<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/01-pyAMPACT_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Introduction</h1>

In [1]:
!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git
from IPython.utils import io
print('Importing libraries...')
with io.capture_output() as captured:
    !pip install --upgrade pandas
    !pip install -i https://test.pypi.org/simple/ --no-deps pyampact
    import pyampact
import pandas as pd
import numpy as np
import librosa

Cloning into 'pyAMPACTtutorials'...
remote: Enumerating objects: 430, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 430 (delta 154), reused 181 (delta 106), pack-reused 177
Receiving objects: 100% (430/430), 19.44 MiB | 9.45 MiB/s, done.
Resolving deltas: 100% (215/215), done.
Importing libraries...


### Load audio with librosa and symbolic files with pyAMPACT's `Score` function. Audio files can be any format supported by librosa (wav, mp3, flac, ogg, etc.) and symbolic files can be any format supported by music21 (kern, xml, mei, midi, abc, etc.).

In [2]:
audio_file = '/content/pyAMPACTtutorials/test_files/aveMaria_seg.wav'
score_file = '/content/pyAMPACTtutorials/test_files/aveMaria_seg.mei'

y, original_sr = librosa.load(audio_file)

piece = pyampact.Score(score_file)

### Run the alignment with pyampact's `run_alignment` function.



In [3]:
# You can specify the target sample rate (target_sr), window size in
# milliseconds (win_ms), the hop size (hop_length) as well as arguments
# that dictate the way that the symbolic file is represented as a
# spectrogram-like mask: the bin width of each note/harmonic (width)
# and the number of harmonics (n_harm).
target_sr = 4000
win_ms = 100
hop_length = 32
width = 3
n_harm = 3
import pdb
res, dtw, spec, nmat = pyampact.run_alignment(
    y, original_sr, piece, piece.nmats(), width, target_sr, n_harm, win_ms, hop_length)

### Estimate and compile performance parameters

In [4]:
# Data compilation takes the output of the alignment (res and nmat) and the
# original audio file. It uses the time point estimated by the alignment to
# index into an instantaneous frequency representation of the audio to
# estimate fundamental frequency, power, and a spectral representation.
# It then estimates note-wise tuning-, dynamics, and timbre-related descriptors.

performData=pyampact.data_compilation(res, nmat, audio_file)

### Export to Kern and MEI

In [5]:
piece.toKern('toKern.krn')

In [6]:
piece.toMEI('toMEI.mei')